# LocalUISGD Test


In [1]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from data import ImplicitData
import pandas as pd
from BISGD import BISGD
from LocalUBISGD_normal import LocalUBISGD_normal
from LocalUBISGD_normal_03 import LocalUBISGD_normal_03
from LocalUBISGD_normal_05 import LocalUBISGD_normal_05
from LocalUBISGD_normal_08 import LocalUBISGD_normal_08
from collections import defaultdict
from eval_implicit import EvalPrequential
from datetime import datetime
import getopt
print("plz")


plz


## Functions


In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [3]:
sys.path.append('./')
configPath = 'config'
dataPath = 'palco_2010.tsv'


Data = []
data = pd.read_csv("palco_2010.tsv","\t")
data = data.head(n=58800)
print(data)
Data = data.values.tolist()



/home/mmelo/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


       user_id  track_id            timestamp
0         4220     31313  2010-01-01 00:10:36
1         4220     29567  2010-01-01 00:13:07
2         4220     31313  2010-01-01 00:13:18
3         3970     34925  2010-01-01 00:39:21
4         6258     18106  2010-01-01 00:46:25
...        ...       ...                  ...
58795    47350      9040  2010-01-24 11:04:20
58796    47350      9034  2010-01-24 11:04:32
58797    47350      9032  2010-01-24 11:05:09
58798    47350      9030  2010-01-24 11:05:11
58799    47350      9031  2010-01-24 11:05:17

[58800 rows x 3 columns]


# Optimal param for BISGD and IBISGD

## Iterations

In [4]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

iterations = [5,10,15,20,25,30]

for i in iterations:
    print("iter")
    print(i)
    num_clusters = 8
    cl_num_iterations = i
    cl_learn_rate = 0.1
    cl_regularization = 0.1
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update']))    


iter
5
start time 2022-06-27 22:52:25.932615


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.20648521505376344
end time 2022-06-27 23:15:04.958642
run time 0:22:39.026027

get tuple 7.173848800918683e-06
recommend 0.21279635308410533
eval_point 3.1933869405459335e-05
update 0.001555542856657586
iter
10
start time 2022-06-27 23:15:05.094078
npmean(resuls[Recall@20]) 0.2122753475754493
end time 2022-06-27 23:38:14.899564
run time 0:23:09.805486

get tuple 7.451059055977127e-06
recommend 0.21787065132143701
eval_point 3.238005652675066e-05
update 0.0017648179798710102
iter
15
start time 2022-06-27 23:38:15.021250
npmean(resuls[Recall@20]) 0.21140380126708902
end time 2022-06-28 00:01:56.800227
run time 0:23:41.778977

get tuple 7.207519343110169e-06
recommend 0.21794427832431418
eval_point 3.221731418051216e-05
update 0.0019307842789864054
iter
20
start time 2022-06-28 00:01:56.919212
npmean(resuls[Recall@20]) 0.20914246434095204
end time 2022-06-28 00:26:21.614267
run time 0:24:24.695055

get tuple 7.947890125975317e-06
recommend 0.2297127563163861
ev

## Eta

In [5]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])


etas = [0.0001, 0.001, 0.01, 0.1]

for e in etas:
    print("eta")
    print(e)
    num_clusters = 8
    cl_num_iterations = 25
    cl_learn_rate = e
    cl_regularization = 0.1
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

eta
0.0001
start time 2022-06-28 01:13:43.208564
npmean(resuls[Recall@20]) 0.20957482423836626
end time 2022-06-28 01:37:13.401709
run time 0:23:30.193145

get tuple 7.428364688847341e-06
recommend 0.21425547131256473
eval_point 3.2826933846411434e-05
update 0.0021973208180901143
eta
0.001
start time 2022-06-28 01:37:13.542510
npmean(resuls[Recall@20]) 0.21397830951971078
end time 2022-06-28 02:01:02.553167
run time 0:23:49.010657

get tuple 7.744873462080144e-06
recommend 0.22227397997667836
eval_point 3.379744452399176e-05
update 0.002325842781942718
eta
0.01
start time 2022-06-28 02:01:02.690429
npmean(resuls[Recall@20]) 0.22003034901365706
end time 2022-06-28 02:25:40.354471
run time 0:24:37.664042

get tuple 7.930361494726064e-06
recommend 0.22578461209189288
eval_point 3.336835040187498e-05
update 0.002337498198561117
eta
0.1
start time 2022-06-28 02:25:40.483698
npmean(resuls[Recall@20]) 0.2058874160784989
end time 2022-06-28 02:49:17.703635
run time 0:23:37.219937

get tuple 7.

## Lambda

In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

lambdas = [0,0.0001, 0.001, 0.01, 0.1]   

for l in lambdas:
    print("lambda")
    print(l)
    num_clusters = 8
    cl_num_iterations = 25
    cl_learn_rate = 0.001
    cl_regularization = l
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

lambda
0
start time 2022-06-28 02:49:18.219285
npmean(resuls[Recall@20]) 0.20926243567753003
end time 2022-06-28 03:13:26.526700
run time 0:24:08.307415

get tuple 8.078671637035551e-06
recommend 0.22408634467149477
eval_point 3.3380318669471286e-05
update 0.0023943171614692325
lambda
0.0001
start time 2022-06-28 03:13:26.650863
npmean(resuls[Recall@20]) 0.21600274725274726
end time 2022-06-28 03:37:33.302692
run time 0:24:06.651829

get tuple 7.814055397396995e-06
recommend 0.2249065780868897
eval_point 3.439826624734061e-05
update 0.0023082946838975764
lambda
0.001
start time 2022-06-28 03:37:33.431582
npmean(resuls[Recall@20]) 0.20976767848058334
end time 2022-06-28 04:00:57.129230
run time 0:23:23.697648

get tuple 7.496013933298539e-06
recommend 0.2154823048995757
eval_point 3.197271176590967e-05
update 0.0022445950581102955
lambda
0.01
start time 2022-06-28 04:00:57.252712
npmean(resuls[Recall@20]) 0.2159033719174635
end time 2022-06-28 04:24:10.932337
run time 0:23:13.679625

ge

SyntaxError: invalid syntax (2083253283.py, line 1)

# With variance in the number of models

In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

models = [5,10,15,20,25,30]   

for m in models:
    print("model")
    print(m)
    num_clusters = m
    cl_num_iterations = 25
    cl_learn_rate = 0.001
    cl_regularization = 0.01
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

model
5
start time 2022-06-30 02:03:02.740359
npmean(resuls[Recall@20]) 0.18576329331046312
end time 2022-06-30 02:15:27.725461
run time 0:12:24.985102

get tuple 6.608208831475706e-06
recommend 0.1092696620506038
eval_point 2.925007461888099e-05
update 0.0018203735148825612
model
10
start time 2022-06-30 02:15:27.794784
npmean(resuls[Recall@20]) 0.21348893083919684
end time 2022-06-30 02:39:23.023010
run time 0:23:55.228226

get tuple 6.971874204622645e-06
recommend 0.22301633504890794
eval_point 3.08888515395817e-05
update 0.002291893792801163
model
15
start time 2022-06-30 02:39:23.124542
npmean(resuls[Recall@20]) 0.2246900826446281
end time 2022-06-30 03:14:02.924846
run time 0:34:39.800304

get tuple 6.926655769348144e-06
recommend 0.3311868080094498
eval_point 2.9386403475254363e-05
update 0.0026415476385427982
model
20
start time 2022-06-30 03:14:03.055364
npmean(resuls[Recall@20]) 0.23436138700121972
end time 2022-06-30 04:00:15.213136
run time 0:46:12.157772

get tuple 6.86527

## Full data test

In [3]:
sys.path.append('./')
configPath = 'config'
dataPath = 'palco_2010.tsv'


Data = []
data = pd.read_csv("palco_2010.tsv","\t")

Data = data.values.tolist()

Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.001
cl_regularization = 0.0001
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

/home/mmelo/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


start time 2022-09-21 21:38:57.356720


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.196693657219973
end time 2022-09-21 23:31:54.020512
run time 1:52:56.663792

get tuple 9.497223686585538e-06
recommend 0.6848495746591957
eval_point 2.9749675640049573e-05
update 0.004598371962139776


# Variation in overlap_factor

## 0.333333

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.001
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal_03(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

## 0.5

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.001
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal_05(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

## 0.8

In [ ]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.001
cl_regularization = 0.1
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal_08(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

### palco_sample5_1

In [14]:
sys.path.append('./')



Data = []
data = pd.read_csv("palco_sample5_1.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  user_id  track_id            timestamp
0              87     4110     39059  2010-01-01 10:13:30
1              88     4110     36931  2010-01-01 10:14:19
2              89     4110     36360  2010-01-01 10:14:28
3              90     4110     38603  2010-01-01 10:14:31
4              91     4110     39062  2010-01-01 10:18:30
...           ...      ...       ...                  ...
32570      588644    66046     60491  2010-12-31 18:54:15
32571      588649    47201     51413  2010-12-31 18:56:51
32572      588650    66046     57049  2010-12-31 18:58:39
32573      588652    47201     74940  2010-12-31 19:01:51
32574      588658    47201     74798  2010-12-31 19:05:27

[32575 rows x 4 columns]


In [15]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.001
cl_regularization = 0.0001
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:03:22.550533


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.13803680981595093
end time 2022-09-20 19:05:30.502569
run time 0:02:07.952036

get tuple 6.194652506547891e-06
recommend 0.17187089203325517
eval_point 3.338664587290009e-05
update 0.0021979312120298145


### palco_sample5_2

In [16]:
sys.path.append('./')



Data = []
data = pd.read_csv("palco_sample5_2.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  user_id  track_id            timestamp
0               9    40958      8826  2010-01-01 01:22:39
1              10    40958      8825  2010-01-01 01:22:50
2              11    40958      8836  2010-01-01 01:22:58
3              12    40958     19882  2010-01-01 01:23:26
4              13    40958     27698  2010-01-01 01:23:28
...           ...      ...       ...                  ...
27471      588433       66     28216  2010-12-31 17:44:54
27472      588439       66     28217  2010-12-31 17:48:38
27473      588443       66     28218  2010-12-31 17:51:16
27474      588448       66     28216  2010-12-31 17:55:11
27475      588450       66     28217  2010-12-31 17:58:54

[27476 rows x 4 columns]


In [17]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.001
cl_regularization = 0.0001
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:05:31.061245


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.10283687943262411
end time 2022-09-20 19:07:13.882024
run time 0:01:42.820779

get tuple 5.710226931932083e-06
recommend 0.16656345932196218
eval_point 3.2152689940540504e-05
update 0.0020234509139195477


### palco_sample5_3

In [18]:
sys.path.append('./')



Data = []
data = pd.read_csv("palco_sample5_3.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  user_id  track_id            timestamp
0            1908     4401      9058  2010-01-02 02:23:45
1            1912     4401      9058  2010-01-02 02:23:54
2            1914     4401      9058  2010-01-02 02:24:00
3            1917     4401      9061  2010-01-02 02:25:13
4            1938     4401      9060  2010-01-02 02:35:41
...           ...      ...       ...                  ...
30796      588678    68503     74879  2010-12-31 19:37:09
30797      588679    68503     74512  2010-12-31 19:39:50
30798      588680    68503     73508  2010-12-31 19:39:51
30799      588681    68503     70401  2010-12-31 19:39:55
30800      588787    54903      3390  2010-12-31 21:35:26

[30801 rows x 4 columns]


In [19]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.001
cl_regularization = 0.0001
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:07:14.238732


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.12982456140350876
end time 2022-09-20 19:09:17.092262
run time 0:02:02.853530

get tuple 5.82981419253659e-06
recommend 0.2046658432274534
eval_point 3.2211604871247945e-05
update 0.002085535866873605


### palco_sample5_4

In [23]:
sys.path.append('./')



Data = []
data = pd.read_csv("palco_sample5_4.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  user_id  track_id            timestamp
0               5     6258     18106  2010-01-01 00:46:25
1             254    46286     32670  2010-01-01 13:07:15
2             257    46286     36970  2010-01-01 13:08:05
3             323      927     27213  2010-01-01 13:52:23
4             325      927      1123  2010-01-01 13:53:08
...           ...      ...       ...                  ...
31457      588391    67485     66079  2010-12-31 17:20:40
31458      588392    67485     66080  2010-12-31 17:22:07
31459      588396    67485     66081  2010-12-31 17:24:42
31460      588398    67485     66082  2010-12-31 17:27:11
31461      588401    67485     65764  2010-12-31 17:28:08

[31462 rows x 4 columns]


In [24]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.001
cl_regularization = 0.0001
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 20:15:38.965689


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.14482758620689656
end time 2022-09-20 20:17:25.573037
run time 0:01:46.607348

get tuple 5.62492334957492e-06
recommend 0.15182369330833698
eval_point 3.075846310319572e-05
update 0.001979977406843427


### palco_sample5_5

In [21]:
sys.path.append('./')



Data = []
data = pd.read_csv("palco_sample5_5.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  user_id  track_id            timestamp
0              54     3898     35623  2010-01-01 03:33:50
1              55     3898       543  2010-01-01 03:33:57
2              56     3898       545  2010-01-01 03:34:25
3              59     3898       681  2010-01-01 03:35:46
4              73     3898       550  2010-01-01 03:40:30
...           ...      ...       ...                  ...
28204      588645     2704     74092  2010-12-31 18:54:19
28205      588646     2704     74095  2010-12-31 18:54:30
28206      588654     2704     70876  2010-12-31 19:03:19
28207      588805    38854     26042  2010-12-31 21:49:17
28208      588810    38854     24985  2010-12-31 21:57:49

[28209 rows x 4 columns]


In [22]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])

num_clusters = 8
cl_num_iterations = 25
cl_learn_rate = 0.001
cl_regularization = 0.0001
num_factors = 160
num_iter = 4
learn_rate = 0.5
regularization = 0.4
model = LocalUBISGD_normal(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 10)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 100)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 19:09:18.440308


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.11678832116788321
end time 2022-09-20 19:11:05.097364
run time 0:01:46.657056

get tuple 5.860340169256409e-06
recommend 0.17378961169806711
eval_point 3.381628189643804e-05
update 0.0020834200773060625
